In [ ]:
import os
import json
import hashlib
import hmac
from pathlib import Path
from scipy.stats import chisquare, kstest
import matplotlib.pyplot as plt
from tabulate import tabulate

# === SETUP DATA FOLDER ===
data_folder = Path("casino_data")
data_folder.mkdir(exist_ok=True)

# === UTILITY FUNCTIONS ===

def hmac_sha256(key, message):
    return hmac.new(key.encode(), message.encode(), hashlib.sha256).hexdigest()

def generate_fair_roll(server_seed, client_seed, nonce):
    message = f"{client_seed}:{nonce}"
    hmac_hash = hmac_sha256(server_seed, message)
    index = 0
    while True:
        sub_hash = hmac_hash[index:index+5]
        if len(sub_hash) < 5:
            return 0
        roll = int(sub_hash, 16)
        if roll < 1000000:
            break
        index += 1
    return (roll % 10000) / 100

def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

def save_json(path, data):
    with open(path, 'w') as f:
        json.dump(data, f, indent=4)

# === FAIRNESS TEST FUNCTION ===
def run_fairness_test(rolls):
    print("\n=== Fairness Check (Last 20 Rolls) ===")
    if len(rolls) < 20:
        print(" Not enough data to run fairness test.")
        return

    recent_rolls = rolls[-20:]

    ks_stat, ks_p = kstest(recent_rolls, 'uniform', args=(0, 100))
    chi_bins = [0]*10
    for r in recent_rolls:
        chi_bins[int(r//10)] += 1
    chi_stat, chi_p = chisquare(chi_bins, f_exp=[2]*10)  # Expect 2 in each bin for 20 rolls

    print(f" KS-Test: stat={ks_stat:.4f}, p-value={ks_p:.4f} => {'PASS' if ks_p > 0.05 else 'FAIL'}")
    print(f" Chi-Square Test: stat={chi_stat:.2f}, p-value={chi_p:.4f} => {'PASS' if chi_p > 0.05 else 'FAIL'}")

    plt.hist(recent_rolls, bins=10, range=(0, 100), edgecolor='black')
    plt.title("Fairness Check: Roll Distribution")
    plt.xlabel("Roll")
    plt.ylabel("Frequency")
    plt.show()

# === FORCE RESET PLAYER DATA ===
players_path = data_folder / "players.json"
players_data = {
    "tanmay": {"username": "Tanmay", "client_seed": "tanmay_seed", "balance": 1000, "nonce": 1},
    "noor": {"username": "Noor", "client_seed": "noor_seed", "balance": 1000, "nonce": 1}
}
save_json(players_path, players_data)
players = players_data
print("Player data reset.")

# === SERVER SEED ===
server_seed = "very-secret-server-seed"
# hashed_seed = hashlib.sha256(server_seed.encode()).hexdigest()
# Server seed hash print intentionally removed

# === GAME LOOP ===
roll_history = []
game_log = []
rounds_played = 0

while True:
    for username in ["tanmay", "noor"]:
        player = players[username]
        print(f"\n=== {player['username']}'s Turn ===")
        print(f"Balance: ${player['balance']}, Nonce: {player['nonce']}")
        try:
            bet = float(input("Enter your bet amount (or 0 to quit): $"))
            if bet == 0:
                print(f"{player['username']} chose to quit.")
                continue
            if bet <= 0 or bet > player["balance"]:
                raise ValueError("Invalid bet amount.")
        except ValueError as e:
            print(f"Error: {e}")
            continue

        roll = generate_fair_roll(server_seed, player["client_seed"], player["nonce"])
        roll_history.append(roll)
        print(f"Roll: {roll:.2f}")

        if username == "tanmay":
            game_type = "Dice"
            if roll < 50:
                result = "Win"
                print(f"{player['username']} wins ${bet}!")
                player["balance"] += bet
            else:
                result = "Lose"
                print(f"{player['username']} loses ${bet}.")
                player["balance"] -= bet
        else:
            game_type = "Coin Flip"
            result_text = "Heads" if roll < 50 else "Tails"
            print(f"Result: {result_text}")
            if result_text == "Heads":
                result = "Win"
                print(f"{player['username']} wins ${bet}!")
                player["balance"] += bet
            else:
                result = "Lose"
                print(f"{player['username']} loses ${bet}.")
                player["balance"] -= bet

        # Log this round
        game_log.append({
            "Player": player["username"],
            "Game": game_type,
            "Roll": round(roll, 2),
            "Result": result,
            "Bet": bet,
            "Balance": round(player["balance"], 2),
            "Nonce": player["nonce"]
        })

        player["nonce"] += 1
        players[username] = player
        rounds_played += 1

        if rounds_played % 20 == 0:
            run_fairness_test(roll_history)

        save_json(players_path, players)

    # Show balances if asked
    show_balances = input("\nShow balances? (y/n): ").strip().lower()
    if show_balances == 'y':
        print("\n=== Balances ===")
        for name, p in players.items():
            print(f"{p['username']}: ${p['balance']} (Nonce: {p['nonce']})")

    # Show game log table
    if game_log:
        print("\n=== Game Log ===")
        print(tabulate(game_log, headers="keys", tablefmt="fancy_grid"))

    # Continue or exit
    cont = input("\nContinue playing? (y/n): ").strip().lower()
    if cont != 'y':
        break


Player data reset.

=== Tanmay's Turn ===
Balance: $1000, Nonce: 1
Enter your bet amount (or 0 to quit): $200
Roll: 60.98
Tanmay loses $200.0.

=== Noor's Turn ===
Balance: $1000, Nonce: 1
Enter your bet amount (or 0 to quit): $100
Roll: 42.60
Result: Heads
Noor wins $100.0!

Show balances? (y/n): y

=== Balances ===
Tanmay: $800.0 (Nonce: 2)
Noor: $1100.0 (Nonce: 2)

=== Game Log ===
╒══════════╤═══════════╤════════╤══════════╤═══════╤═══════════╤═════════╕
│ Player   │ Game      │   Roll │ Result   │   Bet │   Balance │   Nonce │
╞══════════╪═══════════╪════════╪══════════╪═══════╪═══════════╪═════════╡
│ Tanmay   │ Dice      │  60.98 │ Lose     │   200 │       800 │       1 │
├──────────┼───────────┼────────┼──────────┼───────┼───────────┼─────────┤
│ Noor     │ Coin Flip │  42.6  │ Win      │   100 │      1100 │       1 │
╘══════════╧═══════════╧════════╧══════════╧═══════╧═══════════╧═════════╛

Continue playing? (y/n): y

=== Tanmay's Turn ===
Balance: $800.0, Nonce: 2
Enter your